# California Housing - Exploratory Data Analysis

This notebook performs comprehensive exploratory data analysis on the California Housing dataset.

## Objectives
1. Load and understand the dataset
2. Assess data quality
3. Clean and preprocess data
4. Engineer features
5. Perform statistical analysis
6. Generate visualizations
7. Demonstrate SQL operations
8. Derive insights for modeling

## 1. Setup & Imports

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from pathlib import Path

# Add parent directory to path
sys.path.append('..')

# Import project modules
from src.dataset import HousingDataProcessor
from src.services.database import DatabaseManager
from src.plots import EDAAnalyser
from src.config import *

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Plotting settings
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✅ All imports successful!")

## 2. Data Loading

In [ ]:
# Initialize data processor
processor = HousingDataProcessor()

# Load California housing data
data = processor.load_data()

print(f"\nDataset shape: {data.shape}")
print(f"Rows: {data.shape[0]:,}")
print(f"Columns: {data.shape[1]}")

In [ ]:
# Display first few rows
print("First 5 rows:")
data.head()

In [ ]:
# Display last few rows
print("Last 5 rows:")
data.tail()

In [ ]:
# Dataset info
print("Dataset Information:")
data.info()

## 3. Data Quality Assessment

In [ ]:
# Check for missing values
print("Missing Values Check:")
missing = processor.check_missing_values()

if len(missing) == 0:
    print("\n✅ No missing values found!")
else:
    print(f"\n⚠️ Found missing values in {len(missing)} columns")

In [ ]:
# Check for duplicates
duplicates = data.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

if duplicates == 0:
    print("✅ No duplicates found!")
else:
    print(f"⚠️ Found {duplicates} duplicate rows")

In [ ]:
# Check data types
print("Data Types:")
print(data.dtypes)

## 4. Data Cleaning

In [ ]:
# Handle missing values
print("Handling missing values...")
cleaned_data = processor.handle_missing_values(strategy='median')

print(f"\nBefore: {data.shape[0]:,} rows")
print(f"After: {cleaned_data.shape[0]:,} rows")

In [ ]:
# Remove outliers
print("Removing outliers using IQR method...")
cleaned_data = processor.remove_outliers(method='iqr', threshold=1.5)

print(f"\nFinal cleaned dataset: {cleaned_data.shape[0]:,} rows")
print(f"Removed: {data.shape[0] - cleaned_data.shape[0]:,} rows ({((data.shape[0] - cleaned_data.shape[0])/data.shape[0]*100):.2f}%)")

## 5. Feature Engineering

In [ ]:
# Apply feature engineering
print("Applying feature engineering...")
engineered_data = processor.apply_feature_engineering()

print(f"\nOriginal features: {cleaned_data.shape[1]}")
print(f"With engineered features: {engineered_data.shape[1]}")
print(f"\nNew features added: {engineered_data.shape[1] - cleaned_data.shape[1]}")

In [ ]:
# Display engineered features
print("Engineered Features:")
new_cols = [col for col in engineered_data.columns if col not in data.columns]
print(new_cols)

# Show sample of engineered features
engineered_data[['rooms_per_household', 'bedrooms_per_room', 
                'population_per_household', 'income_category', 'age_category']].head(10)

In [ ]:
# Validate engineered features
print("Validating feature calculations...")

# Check rooms per household
sample_idx = 0
calculated_rooms = engineered_data.loc[sample_idx, 'total_rooms'] / engineered_data.loc[sample_idx, 'households']
stored_rooms = engineered_data.loc[sample_idx, 'rooms_per_household']

print(f"Sample validation for rooms_per_household:")
print(f"  Calculated: {calculated_rooms:.2f}")
print(f"  Stored: {stored_rooms:.2f}")
print(f"  Match: {np.isclose(calculated_rooms, stored_rooms)}")

## 6. Univariate Analysis

In [ ]:
# Summary statistics
print("Summary Statistics:")
engineered_data.describe()

In [ ]:
# Target variable distribution
print("Target Variable (median_house_value) Statistics:")
target_stats = engineered_data['median_house_value'].describe()
print(target_stats)

print(f"\nRange: ${target_stats['min']:,.0f} - ${target_stats['max']:,.0f}")
print(f"Mean: ${target_stats['mean']:,.0f}")
print(f"Median: ${target_stats['50%']:,.0f}")

In [ ]:
# Distribution of each numeric feature
numeric_cols = engineered_data.select_dtypes(include=[np.number]).columns

print("Numeric Features:")
for col in numeric_cols:
    print(f"\n{col}:")
    print(f"  Mean: {engineered_data[col].mean():.2f}")
    print(f"  Median: {engineered_data[col].median():.2f}")
    print(f"  Std: {engineered_data[col].std():.2f}")

## 7. Bivariate Analysis

In [ ]:
# Correlation with target variable
target_corr = engineered_data.select_dtypes(include=[np.number]).corr()['median_house_value'].sort_values(ascending=False)

print("Correlations with Median House Value:")
print("="*50)
print(target_corr)
print("="*50)

In [ ]:
# Top 5 positive correlations
print("Top 5 Features with Highest Positive Correlation:")
top_5_positive = target_corr[target_corr < 1.0].head(5)
print(top_5_positive)

In [ ]:
# Top 5 negative correlations
print("Top 5 Features with Highest Negative Correlation:")
top_5_negative = target_corr.tail(5)
print(top_5_negative)

## 8. Multivariate Analysis

In [ ]:
# Initialize EDA Analyser
print("Initializing EDA Analyser...")
eda = EDAAnalyser(engineered_data)

print("\nGenerating all visualizations...")
print("This may take a few moments...")

In [ ]:
# Generate all 10 visualizations
plots = eda.generate_all_plots()

print(f"\n✅ Generated {len(plots)} visualizations!")
print(f"Saved to: {FIGURES_DIR}")

for plot_name, path in plots.items():
    print(f"  - {plot_name}: {Path(path).name}")

In [ ]:
# Display correlation analysis
print("Detailed Correlation Analysis:")
correlations = eda.get_correlation_analysis('median_house_value')

## 9. SQL Demonstrations

In [ ]:
# Initialize database manager
print("Initializing Database Manager...")
db = DatabaseManager()

# Create tables
print("\nCreating database tables...")
db.create_tables()

In [ ]:
# Insert data into database
print("Inserting data into database...")
db.insert_data(engineered_data, 'housing')

print("\nPopulating district summary table...")
db.populate_district_summary()

In [ ]:
# Demonstration 1: WHERE clause filtering
print("=" * 60)
print("SQL Demonstration 1: WHERE Clause")
print("=" * 60)

print("\nQuery: Filter houses with income between $30k-$50k")
filtered = db.filter_by_income(3.0, 5.0)

print(f"\nFound {len(filtered):,} records")
print("\nSample results:")
filtered[['longitude', 'latitude', 'median_income', 'median_house_value']].head(10)

In [ ]:
# Demonstration 2: GROUP BY aggregation
print("=" * 60)
print("SQL Demonstration 2: GROUP BY Aggregation")
print("=" * 60)

print("\nQuery: Aggregate statistics by income category")
aggregated = db.aggregate_by_income_category()

print("\nAggregated Results:")
aggregated

In [ ]:
# Demonstration 3: INNER JOIN
print("=" * 60)
print("SQL Demonstration 3: INNER JOIN")
print("=" * 60)

print("\nQuery: Join housing with district summary")
joined = db.join_housing_with_summary(limit=20)

print(f"\nJoined {len(joined):,} records")
print("\nSample results showing individual houses vs district averages:")
joined[['income_category', 'median_house_value', 'district_avg_value', 
        'rooms_per_household', 'district_avg_rooms']]

In [ ]:
# Database statistics
print("Database Statistics:")
stats = db.get_statistics()

print(f"\nHousing records: {stats['housing_count']:,}")
print(f"Summary records: {stats['summary_count']:,}")
print(f"Database size: {stats['database_size']:.2f} MB")
print(f"Tables: {stats['tables']}")

## 10. Key Insights & Conclusions

In [ ]:
print("="*60)
print("KEY INSIGHTS FROM EXPLORATORY DATA ANALYSIS")
print("="*60)

# Top correlations
print("\n1. TOP 3 FEATURES CORRELATED WITH HOUSE PRICES:")
top_3 = target_corr[target_corr < 1.0].head(3)
for i, (feature, corr) in enumerate(top_3.items(), 1):
    print(f"   {i}. {feature}: {corr:.3f}")

# Geographic patterns
print("\n2. GEOGRAPHIC PATTERNS:")
print(f"   - Latitude range: {engineered_data['latitude'].min():.2f} to {engineered_data['latitude'].max():.2f}")
print(f"   - Longitude range: {engineered_data['longitude'].min():.2f} to {engineered_data['longitude'].max():.2f}")
print(f"   - Data covers California state")

# Income impact
print("\n3. INCOME IMPACT:")
income_groups = aggregated.sort_values('avg_house_value', ascending=False)
for _, row in income_groups.iterrows():
    print(f"   - {row['income_category'].title()}: ${row['avg_house_value']:,.0f} (n={row['count_districts']:,})")

# Housing characteristics
print("\n4. HOUSING CHARACTERISTICS:")
print(f"   - Average rooms per household: {engineered_data['rooms_per_household'].mean():.2f}")
print(f"   - Average bedrooms per room: {engineered_data['bedrooms_per_room'].mean():.2f}")
print(f"   - Average population per household: {engineered_data['population_per_household'].mean():.2f}")

# Recommendations
print("\n5. RECOMMENDATIONS FOR MODELING:")
print(f"   ✓ Use median_income as primary predictor (correlation: {target_corr['median_income']:.3f})")
print(f"   ✓ Include geographic features (latitude, longitude)")
print(f"   ✓ Leverage engineered features (rooms_per_household, etc.)")
print(f"   ✓ Consider non-linear relationships for income")
print(f"   ✓ Account for outliers in house values")

print("\n" + "="*60)

## 11. Data Export

In [ ]:
# Save processed data
print("Saving processed data...")
processor.save_data(engineered_data, 'processed')

print("\n✅ All data exported successfully!")
print(f"\nProcessed data available at: {PROCESSED_DATA_PATH}")
print(f"Database available at: {DATABASE_PATH}")
print(f"Visualizations available at: {FIGURES_DIR}")

## Summary

This notebook has completed comprehensive exploratory data analysis on the California Housing dataset:

✅ **Data Loading**: Loaded 20,640 housing records with 8 original features

✅ **Data Quality**: Checked for missing values, duplicates, and outliers

✅ **Data Cleaning**: Handled missing values and removed outliers

✅ **Feature Engineering**: Created 5 new features (rooms_per_household, bedrooms_per_room, population_per_household, income_category, age_category)

✅ **Statistical Analysis**: Performed univariate, bivariate, and multivariate analysis

✅ **Visualizations**: Generated 10 different types of plots

✅ **SQL Operations**: Demonstrated WHERE, GROUP BY, and INNER JOIN queries

✅ **Insights**: Identified key patterns and relationships

**Next Steps**:
1. Train linear regression model using processed data
2. Evaluate model performance
3. Make predictions using the Streamlit dashboard

**Files Generated**:
- Processed CSV: `data/processed/housing_processed.csv`
- SQLite Database: `data/housing.db`
- Visualizations: `reports/figures/*.png` (10 plots)